
#### 실제 파이썬 객체와 동일하게 동작하는 사용자 정의 클래스를 만들어보자. 여러 파이썬 객체에서 흔히 볼 수 있는 다음 지 특별 메서드를 구현하도록 한다.

- repr(), bytes() 등 객체를 다른 방식으로 표현하는 내장 함수의 지원
- 클래스 메서드로 대안 생성자 구현
- format() 내장 함수와 str.format() 메서드에서 사용하는 포맷 언어 확장
- 읽기 전용 접근만 허용하는 속성 제공
- 집합 및 딕셔너리 키로 사용할 수 있도록 객체를 해시 가능하게 만들기
- __slots__를 이용해서 메모리 절약하기
- 언제, 어떻게 @classmethod와 @staticmethod 데커레이터를 사용할 것인가
- 파이썬에서 비공개 및 보호된 속성: 사용법, 관례, 한계

# 9.1 객체 표현 
- repr() : 객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환
- str() : 객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환
- 이들을 지원하려면 __repr__()과 __str__() 특별 메서드를 구현해야 한다. 그외에 __bytes__()와 __format__() 도 있다.

# 9.2 벡터 클래스의 부활 

In [1]:
from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y) # 미리 실수로 변환하는 센스
        
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [2]:
v1 = Vector2d(3,4)
v1

Vector2d(3.0, 4.0)

In [3]:
print(v1.x, v1.y)

3.0 4.0


In [4]:
x,y=v1 # __iter__ 의 영향
print(x,y)

3.0 4.0


In [5]:
v1_clone = eval(repr(v1))
v1_clone, v1 == v1_clone # __eq__

(Vector2d(3.0, 4.0), True)

In [6]:
print(v1), repr(v1) # __str__, __repr__

(3.0, 4.0)


(None, 'Vector2d(3.0, 4.0)')

In [7]:
octets = bytes(v1) # __byte__ , 이진표현 생성
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [9]:
abs(v1), bool(v1), bool(Vector2d(0,0)) # __abs__ : 객체의 크기, __bool__ : 객체의 크기가 0일때 False 

(5.0, True, False)

# 9.3 대안 생성자

In [10]:
# bytes를 Vector2d로 변환하는 메서드 추가
from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y) # 미리 실수로 변환하는 센스
        
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod # 클래스 메서드
    def frombytes(cls, octets): # self 매개변수가 없고 대신 자신이 cls로 전달됨
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

v2 = Vector2d.frombytes(b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@')
v2

Vector2d(3.0, 4.0)


# 9.4 @classmethod와 @staticmethod
- @classmethod는 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의하며 클래스 자체를 첫번째 인수로 받게 만든다. 
- frombytes() 같은 대안 생성자를 구현하기 위해 사용 
- 위의 예제에서는 마지막 문장에서 cls(*memv)를 호출해서 객체를 생성한다.

- @staticmethod 데커레이터는 메서드가 특별한 첫번째 인수를 받지 않도록 메서드를 변경한다. 

In [11]:
class Demo:
    @classmethod
    def klassmeth(*args): # *args 나 아예 인수를 넣지 않는 경우는 자동으로 파이썬에서 cls를 첫번쨰 인수로 추가하는듯
        return args
    @staticmethod
    def statmeth(*args):
        return args

In [12]:
Demo.klassmeth(), Demo.klassmeth('spam')

((__main__.Demo,), (__main__.Demo, 'spam'))

In [13]:
Demo.statmeth(), Demo.statmeth('spam')

((), ('spam',))

# 9.5 포맷된 출력

In [2]:
print(format(1/2.43, '0.4f'))
print(format(42, 'b'))
print(format(2/3, '.1%'))

from datetime import datetime 
now = datetime.now()
format(now, '%H:%M:%S')

0.4115
101010
66.7%


'05:22:57'

In [3]:
# 벡터 포멧 
from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y) # 미리 실수로 변환하는 센스
        
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod # 클래스 메서드
    def frombytes(cls, octets): # self 매개변수가 없고 대신 자신이 cls로 전달됨
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle)
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({},{})'

        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    #극좌표
    def angle(self):
        return math.atan2(self.y, self.x)

format(Vector2d(1,1), 'p')

'<1.4142135623730951, <bound method Vector2d.angle of Vector2d(1.0, 1.0)>>'

# 9.6 해시 가능한 Vector2d

In [4]:
#해시가 불가능하고, 집합으로 사용 불가
v1 = Vector2d(3,4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [7]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    # 해시 메서드, xor 연산
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [8]:
v1, v2 = Vector2d(3, 4), Vector2d(3.2, 4.1)
print(hash(v1), hash(v2))
print(set([v1,v2]))

7 384307168202283015
{Vector2d(3.2, 4.1), Vector2d(3.0, 4.0)}


In [9]:
# name mangling
v1 = Vector2d(3, 4)
print(v1.__dict__)
print(v1._Vector2d__x)

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}
3.0


In [ ]:
# __slot__ : dict 대신 tuple 에 저장
class Vector2d:
    __slots__ = ('__x', '__y')
    typecode = 'd'

# 상속할때마다 재정의 해야함 
# 변경할 수 없는 부작용 